# Picture to Plate

In [ ]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor, BitsAndBytesConfig
import torch
from qwen_vl_utils import process_vision_info
import nltk
from rouge_score import rouge_scorer

W0529 15:47:29.450000 3444 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [2]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Set False for 8-bit
    bnb_4bit_compute_dtype=torch.float16
)

In [3]:
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct", torch_dtype="auto", device_map="auto", quantization_config=bnb_config,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


#### **Setup**

In "filename" enter the filename from "Test_Images" folder. In "dish" enter the title of the dish (noisy title)

In [103]:
filename = "Dosa"
image_path = f"./Test_Images/{filename}.jpeg"
dish = "Dosa"

## Description Generation

In [104]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": image_path,
            },
            {"type": "text", "text": f"Describe the food image with ingredients, texture etc present. Dish: {dish}"},
        ],
    }
]

In [105]:
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

In [106]:
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
description_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
description_text = description_text[0]
print(description_text)

The dish in the image is a dosa, which is a type of flatbread typically made from rice and urad dal (black gram) flour. Here's a detailed description of the ingredients and textures present:

Ingredients:
:
- A dosa roll on the left side, likely made from rice and urad dal flour.
- A plate containing several small bowls filled with various dipping sauces or accompaniments.

These include:
- Green chutney (likely made from green peas)
- Yellow chutney (possibly made from chickpea or lentils)
- Red chutney (likely made from red lentils or t


## Recipe Generation

**Prompt Structure**

In [116]:
recipe_generator = f"""You are a professional recipe generator.
Dish: {dish}
Description of the food : {description_text}

Format: The recipe should be in "3-steps" only. 
1. [steps]
2. [steps]
3. [steps]

Examples: 
Dish: Mustard Fish
Summary: 1. Clean and slice the fish. Soak mustard seeds for 2 hours, then grind with salt and green chillies to a paste. Mix in grated coconut, yoghurt, sugar, turmeric, and mustard oil to make a marinade.
2. Coat the fish pieces with the mixture, place them in a tiffin box with halved green chillies on top, fasten the lid, and marinate for at least 15 minutes.
3. Steam the sealed tiffin in a covered pan with hot water for 15 minutes. Let it rest for 5 minutes before serving.

Dish: Butter Chicken
Summary: 1. Marinate boneless chicken pieces in yogurt, lemon juice, ginger-garlic paste, chili powder, and garam masala for at least 1 hour. Then grill, bake, or pan-fry until cooked and slightly charred.
2. In a pan, heat butter and sauté ginger, garlic, and tomatoes until soft. Blend this into a smooth puree, then return to the pan and add cream, kasuri methi (dried fenugreek), chili powder, and salt. Simmer until rich and creamy.
3. Add the cooked chicken to the sauce, simmer for 10–15 minutes, adjust seasoning, and serve hot with naan or rice.

Dish: Cheesy Bake
Summary: 1. Boil or steam your choice of veggies (e.g., broccoli, cauliflower, carrots, corn) or cooked pasta/chicken. Lightly season with salt, pepper, and herbs.
2. In a pan, melt butter, add flour, and cook for a minute. Slowly whisk in milk to make a smooth white sauce. Add grated cheese (like cheddar or mozzarella) and stir until melted and creamy.
3. Mix the base with the cheese sauce, pour into a greased baking dish, top with more cheese, and bake at 180°C (350°F) for 20–25 minutes until golden and bubbly. Serve hot!"""

In [117]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": image_path,
            },
            {"type": "text", "text": recipe_generator},
        ],
    }
]

In [118]:
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
).to("cuda")

In [119]:
generated_ids = model.generate(
    **inputs,
    max_new_tokens=300,
    ).to("cpu")
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
recipe_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(recipe_text[0])
generated_recipe = recipe_text[0]


Dish: Dosa
Summary:
1. Roll out the dosa batter using a rolling pin until it forms a thin, round circle. Place the rolled-out dosa on a hot griddle or skillet and cook for about 2-3 minutes on each side until golden brown and crispy.
2. While the dosa is cooking, prepare the chutneys and sauces by mixing the ingredients together. You can use green peas, chickpeas, red lentils, or any other vegetables you prefer.
3. Once the dosa is fully cooked, place it on a plate and top it with the prepared chutneys and sauces. Serve hot and enjoy the delicious dosa with its variety of flavors and textures.


## ROUGE Score

In [120]:
reference_summary_path = f"./Test_Summaries/{filename}.txt"
with open(reference_summary_path, "r", encoding="utf-8") as ref_file:
    reference_summary = ref_file.read().strip()

In [121]:
scorer = rouge_scorer.RougeScorer(
    ['rouge1', 'rouge2', 'rougeL'], 
    use_stemmer=True
)

In [122]:
scores = scorer.score(reference_summary, generated_recipe)

In [123]:
print("ROUGE Scores:")
for metric, score in scores.items():
    print(f"{metric.upper()}:")
    print(f"  Precision: {score.precision:.4f}")
    print(f"  Recall:    {score.recall:.4f}")
    print(f"  F1:        {score.fmeasure:.4f}")
    print()

ROUGE Scores:
ROUGE1:
  Precision: 0.3243
  Recall:    0.4737
  F1:        0.3850

ROUGE2:
  Precision: 0.0909
  Recall:    0.1333
  F1:        0.1081

ROUGEL:
  Precision: 0.2072
  Recall:    0.3026
  F1:        0.2460

